In [18]:
import nltk 
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from nltk.stem import WordNetLemmatizer
import re
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [19]:
# Read the data
a=[]
b=[]
with open('/content/drive/MyDrive/Queens Collage/NLP Course 2022/SMSSpamCollection.txt','r') as f:
    l=f.readlines()
    for j in l:
        a.append(j.split('\t')[0])
        b.append(j.split('\t')[1])

In [20]:
# convert to pandas 
d={'label':a,'document':b}
df=pd.DataFrame(d)
df.head()

,label,document
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...\n
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [21]:
# taking a sample of data
sample_data = df.tail(574)

In [22]:
# get stop words
# from nltk.corpus import stopwords
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [23]:
# lower case

sample_data1 = (
    # start with training data
    sample_data
    # convert to lower case
    .assign(clean_document= lambda x:[text.lower() for text in x.document]) 
    # remove punctuation
    .assign(clean_document= lambda x: [re.sub("[^\w\s]","",text) for text in x.clean_document])
     # remove numbers
    .assign(clean_document= lambda x: [re.sub("\d","",text) for text in x.clean_document])
    # remove leading & trainling spaces 
    .assign(clean_document = lambda x: [text.strip() for text in x.clean_document])
    # remove all the '\n' values with space
    .assign(clean_document= lambda x: [re.sub(r"\n","",text) for text in x.clean_document] )
    # word tokenize
    .assign(word_token= lambda x: [nltk.word_tokenize(text) for text in x.clean_document] )
    # # remove stop words
    .assign(word_token= lambda x: [list(set(list_of_words).difference(stop_words)) for list_of_words in x.word_token])
    # Lemmatising the words
    # .assign(word_token= lambda x: [[lem.lemmatize(a_word) for a_word in list_of_words] for list_of_words in x.word_token ])
    #  # input to sklearn
    .assign(document_to_sklearn= lambda x: [" ".join(map(str,list_of_words)) for list_of_words in x.word_token ])
)

In [24]:
from collections import defaultdict

def co_occurrence(sentences, window_size):
    d = defaultdict(int)
    vocab = set()
    for text in sentences:
        # preprocessing (use tokenizer instead)
        text = text.lower().split()
        # iterate over sentences
        for i in range(len(text)):
            token = text[i]
            vocab.add(token)  # add to vocab
            next_token = text[i+1 : i+1+window_size]
            for t in next_token:
                key = tuple( sorted([t, token]) )
                d[key] += 1
    
    # formulate the dictionary into dataframe
    vocab = sorted(vocab) # sort vocab
    df = pd.DataFrame(data=np.zeros((len(vocab), len(vocab)), dtype=np.int16),
                      index=vocab,
                      columns=vocab)
    for key, value in d.items():
        df.at[key[0], key[1]] = value
        df.at[key[1], key[0]] = value
    return df

In [31]:
text = sample_data1['document_to_sklearn']
df = co_occurrence(text, 3)
df

,_,aah,able,absolutely,accent,access,accident,accidentally,accommodationvouchers,account,...,youve,yoville,yr,yrs,yuo,yup,zebra,zed,zeros,ü
_,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aah,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
able,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
absolutely,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
accent,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
yup,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zebra,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zed,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zeros,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
from sklearn.decomposition import PCA
pca = PCA(2)
pca_d = pca.fit_transform(df)
df1 = pd.DataFrame(pca_d,index=df.index,columns=['pca1','pca2'])

In [10]:
df1

,pca1,pca2
_,-0.251599,-0.128430
aah,-0.388955,-0.001953
able,-0.057045,-0.201698
absolutely,-0.299316,-0.097449
accent,-0.330391,-0.068211
...,...,...
yup,0.933648,0.796501
zebra,-0.406122,0.003754
zed,-0.003268,-0.035374
zeros,-0.236534,-0.078678


In [11]:
import plotly.express as px

In [12]:
df1.index

Index(['_', 'aah', 'able', 'absolutely', 'accent', 'access', 'accident',
       'accidentally', 'accommodationvouchers', 'account',
       ...
       'youve', 'yoville', 'yr', 'yrs', 'yuo', 'yup', 'zebra', 'zed', 'zeros',
       'ü'],
      dtype='object', length=2109)

In [ ]:
fig = px.scatter( x=df1["pca1"], y=df1["pca2"],color=df.index)
fig.show()

In [ ]:
# build a function, that , given a word , returns the closest word....